In [2]:
import pyspark
print(pyspark.__version__)

3.1.2


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print(spark.version)


25/02/12 14:38:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 14:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


3.1.2


In [6]:
from pyspark.sql import SparkSession

# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS JSON File Reading") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()



25/02/14 13:33:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/14 13:33:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/02/14 13:33:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
# HDFS에서 JSON 파일 읽기
file_path = "hdfs://localhost:9000/shared_data/label_data/1.Car/1.horn_of_car"

In [8]:
df = spark.read.json(file_path, multiLine= True)

In [9]:
df.printSchema()

root
 |-- annotations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- area: struct (nullable = true)
 |    |    |    |-- end: double (nullable = true)
 |    |    |    |-- start: double (nullable = true)
 |    |    |-- categories: struct (nullable = true)
 |    |    |    |-- category_01: string (nullable = true)
 |    |    |    |-- category_02: string (nullable = true)
 |    |    |    |-- category_03: string (nullable = true)
 |    |    |-- decibel: long (nullable = true)
 |    |    |-- labelName: string (nullable = true)
 |    |    |-- soundQuality: string (nullable = true)
 |    |    |-- subCategory: string (nullable = true)
 |-- audio: struct (nullable = true)
 |    |-- bitRate: string (nullable = true)
 |    |-- duration: double (nullable = true)
 |    |-- fileFormat: string (nullable = true)
 |    |-- fileName: string (nullable = true)
 |    |-- fileSize: long (nullable = true)
 |    |-- recodingType: string (nullable = true)
 |    |-- sample

In [11]:
df.show()

+---------------------+--------------------+-----------------------------+--------------------+--------------------+
|          annotations|               audio|                  environment|                info|             license|
+---------------------+--------------------+-----------------------------+--------------------+--------------------+
| [{{10.55, 9.99}, ...|{705kbps, 11.584,...|         {iPHONE 12 mini, ...|{IMR, 2021-09-20,...|{CC 0, https://ww...|
| [{{16.62, 16.05},...|{705kbps, 22.4, w...|         {iPHONE 12 mini, ...|{IMR, 2021-09-20,...|{CC 0, https://ww...|
| [{{7.401, 2.0}, {...|{1411kbps, 9.401,...|  {갤럭시S6Tab, 자연적, 제...|{IMR, 2021-09-04,...|{CC 0, https://ww...|
| [{{4.13, 2.68}, {...|{1411kbps, 7.178,...|  {갤럭시S6Tab, 자연적, 제...|{IMR, 2021-09-04,...|{CC 0, https://ww...|
| [{{12.944, 2.0}, ...|{705kbps, 14.944,...|        {TASCAM DR-05X, 자...|{IMR, 2021-10-26,...|{CC 0, https://ww...|
| [{{5.48, 3.11}, {...|{1411kbps, 8.733,...|{갤럭시탭S6, 자연적, 제작,...|{IMR, 2021-09-

In [9]:
df.createOrReplaceTempView("json_table")

In [10]:
spark.sql("SELECT audio.fileName, audio.duration, info.contributor FROM json_table").show(truncate=False)

+------------------+--------+-----------+
|fileName          |duration|contributor|
+------------------+--------+-----------+
|1.자동차_19863.wav|11.584  |IMR        |
|1.자동차_19854.wav|22.4    |IMR        |
|1.자동차_133.wav  |9.401   |IMR        |
|1.자동차_197.wav  |7.178   |IMR        |
|1.자동차_30826.wav|14.944  |IMR        |
|1.자동차_203.wav  |8.733   |IMR        |
|1.자동차_30737.wav|10.539  |IMR        |
|1.자동차_30816.wav|10.557  |IMR        |
|1.자동차_30880.wav|10.581  |IMR        |
|1.자동차_108.wav  |5.884   |IMR        |
|1.자동차_18.wav   |8.792   |IMR        |
|1.자동차_191.wav  |6.366   |IMR        |
|1.자동차_214.wav  |10.592  |IMR        |
|1.자동차_224.wav  |6.453   |IMR        |
|1.자동차_262.wav  |6.981   |IMR        |
|1.자동차_30682.wav|13.314  |IMR        |
|1.자동차_30686.wav|17.167  |IMR        |
|1.자동차_30687.wav|12.479  |IMR        |
|1.자동차_30688.wav|14.261  |IMR        |
|1.자동차_30691.wav|13.124  |IMR        |
+------------------+--------+-----------+
only showing top 20 rows



In [35]:
spark.sql("SELECT DISTINCT environment.weather  FROM json_table").show(10,truncate=False)

+-------+
|weather|
+-------+
|맑음   |
|흐림   |
+-------+



In [ ]:
spark.sql("SELECT audio.fileName, audio.duration, info.contributor FROM json_table").show(truncate=False)

In [37]:
spark.sql("""
    SELECT 
        audio.fileName, 
        audio.duration, 
        audio.recodingType, 
        annotations.subCategory
    FROM json_table
    LIMIT 50
""").show()

+------------------+--------+------------+------------+
|          fileName|duration|recodingType| subCategory|
+------------------+--------+------------+------------+
|1.자동차_19863.wav|  11.584|        Mono|[소형차경적]|
|1.자동차_19854.wav|    22.4|        Mono|[소형차경적]|
|  1.자동차_133.wav|   9.401|      Stereo|[대형차경적]|
|  1.자동차_197.wav|   7.178|      Stereo|[소형차경적]|
|1.자동차_30826.wav|  14.944|        Mono|[소형차경적]|
|  1.자동차_203.wav|   8.733|      Stereo|[대형차경적]|
|1.자동차_30737.wav|  10.539|        Mono|[소형차경적]|
|1.자동차_30816.wav|  10.557|        Mono|[소형차경적]|
|1.자동차_30880.wav|  10.581|        Mono|[소형차경적]|
|  1.자동차_108.wav|   5.884|      Stereo|[대형차경적]|
|   1.자동차_18.wav|   8.792|      Stereo|[소형차경적]|
|  1.자동차_191.wav|   6.366|      Stereo|[소형차경적]|
|  1.자동차_214.wav|  10.592|      Stereo|[소형차경적]|
|  1.자동차_224.wav|   6.453|      Stereo|[소형차경적]|
|  1.자동차_262.wav|   6.981|      Stereo|[소형차경적]|
|1.자동차_30682.wav|  13.314|        Mono|[소형차경적]|
|1.자동차_30686.wav|  17.167|        Mono|[소형차경적]|
|1.자동차_30687.wav

In [38]:
result_df = spark.sql("""
    SELECT 
        fileName, 
        duration, 
        recodingType, 
        subCategory
    FROM json_table
    LIMIT 50
""")

result_df.show()

AnalysisException: cannot resolve '`fileName`' given input columns: [json_table.annotations, json_table.audio, json_table.environment, json_table.info, json_table.license]; line 3 pos 8;
'GlobalLimit 50
+- 'LocalLimit 50
   +- 'Project ['fileName, 'duration, 'recodingType, 'subCategory]
      +- SubqueryAlias json_table
         +- Relation[annotations#0,audio#1,environment#2,info#3,license#4] json


In [40]:
result_df = spark.sql("""
    SELECT 
        json_table.audio.fileName, 
        json_table.audio.duration, 
        json_table.audio.recodingType
    FROM json_table
    LIMIT 50
""")

result_df.show()


+------------------+--------+------------+
|          fileName|duration|recodingType|
+------------------+--------+------------+
|1.자동차_19863.wav|  11.584|        Mono|
|1.자동차_19854.wav|    22.4|        Mono|
|  1.자동차_133.wav|   9.401|      Stereo|
|  1.자동차_197.wav|   7.178|      Stereo|
|1.자동차_30826.wav|  14.944|        Mono|
|  1.자동차_203.wav|   8.733|      Stereo|
|1.자동차_30737.wav|  10.539|        Mono|
|1.자동차_30816.wav|  10.557|        Mono|
|1.자동차_30880.wav|  10.581|        Mono|
|  1.자동차_108.wav|   5.884|      Stereo|
|   1.자동차_18.wav|   8.792|      Stereo|
|  1.자동차_191.wav|   6.366|      Stereo|
|  1.자동차_214.wav|  10.592|      Stereo|
|  1.자동차_224.wav|   6.453|      Stereo|
|  1.자동차_262.wav|   6.981|      Stereo|
|1.자동차_30682.wav|  13.314|        Mono|
|1.자동차_30686.wav|  17.167|        Mono|
|1.자동차_30687.wav|  12.479|        Mono|
|1.자동차_30688.wav|  14.261|        Mono|
|1.자동차_30691.wav|  13.124|        Mono|
+------------------+--------+------------+
only showing top 20 rows



In [5]:
spark.stop()